# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['fp16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，其卦象由三个阳爻夹一个阴爻构成，象征着天、云和雷等自然现象。乾卦的含义包括力量、刚强、坚定、行动和领导等。

乾卦的五行属性为木，代表着春季、树木、成长和发展等。在中医和五行理论中，乾卦与肝、脾、心脏和头部等器官相对应。

在八宫图中，乾卦位于北方，与事业、努力和坚持等有关。人们可以通過观察乾卦的位置和与其他卦的相对位置，来推断未来、了解自己的命运和把握自己的命运。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它的卦象由两个阴爻夹一个阳爻构成，象征着法律、诉讼、争端和矛盾等。以下是讼卦的基本解释：

1. 含义：讼卦意味着诉讼、争端和矛盾。它提醒人们在处理问题和争端时要保持冷静和理性，避免过于冲动和情绪化。
2. 五行属性：讼卦的五行属性为水，代表秋天的气息、河流、湖泊等自然现象。在中医和五行理论中，坎卦与肾脏、智慧、变化和液体等相对应。
3. 在八宫图中，讼卦位于西方，与法律、秩序和制度等有关。它提醒人们要遵守法律和规则，保持社会秩序的稳定。
4. 健康方面：讼卦与肾脏和生殖系统等有关，可能意味着在这方面可能会有一些问题。建议在面对诉讼和争端时，保持冷静和理性，避免过度情绪化，以免影响健康。

总之，讼卦是提醒人们在处理问题和争端时要保持冷静和理性，遵守法律和规则，以维护社会秩序的稳定。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240424_050209"

# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
# training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由六个阳爻组成，象征着天（乾为天）的运行和刚健的品质。在卜问中，乾卦预示着吉祥如意，行事刚毅果敢，具有天的保佑。在卜问中，频遭贞卜者之言，吉利却 also 带有一定的威胁，需要 careful 对待。

乾卦的哲学内涵是：天行刚健，刚健却有所统摄，具有吉利的一面，但也带来一定的威胁。在卜问中，频遭贞卜者之言，表明此卦并不完全吉利，却也提示了吉利和威胁并存的现实，需要我们谨慎行事，不可刚硬。

乾卦的运势特点为：阳爻排列有序，刚健吉利，宜刚硬果敢，具有决断力，时机成熟时，宜积极行动。

乾卦的运势解释为：先扬后平坦，时机成熟时，刚健刚硬，果敢决断，宜积极行动。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，象征刚健强劲的特性。它由两个阳爻（即表示数字1的爻）夹一个阴爻（即表示数字0的爻）组成，具有强烈的刚健强劲之力。乾卦的哲学意义是强调刚健强劲的特性，以及顺应天道的运行。

在卜筮中，乾卦预示着吉祥如意，事业顺利，权力威严，刚健强劲。然而，也需要注意防止过于刚硬，导致矛盾激化。在决策时，应该充分考虑各种因素，坚持正道，确保事业和生活的和谐发展。

乾卦的哲学内涵是刚健强劲，力量无穷，具有领导力和强大的力量。在决策中，要充分考虑各种因素，确保各种力量相互协调，促进事业和生活的和谐发展。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，代表诉讼之卦。它由两个卦相组合而成，上卦是乾（天），下卦是坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦的核心哲学是“争位”，强调争抢职位和权力，以及事理乖舛的状况。

讼卦的卦辞为：“象吉，元吉，利见天现"，意味着讼卦有着吉祥如意，利见天现的征兆。同时，卦中的元吉之象代表腰部受损，需要戒慎行动，避免受损进一步加重。

讼卦的运势特点是：开始时可能顺利，有所收获，但随后会受到副卦的抑制，运费唇舌，事业隆起拱卫，但也有来自外部的压力和损伤。初始阶段容易招惹是非和不顺，需要注意言行举止，避免争端，保持和谐稳定。

讼卦的解析：讼卦是一个带有吉祥征兆的卦象，但是需要注意外部干扰和损伤。事情位进展会有所隆起，但也会受到副卦的制约，需要谨慎处理，避免引起争端。

讼卦的运势解析：初始阶段可能顺利，有所收获，但随后会受到副卦的制约，运费唇舌，事业隆起拱卫，但也有来自外部的压力和损伤。初始阶段容易招惹是非和不顺，需要注意言行举止，避免争端，保持和谐稳定。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))）：
[gMASK]sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))）：
[gMASK]sop 师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，象征着刚健强劲的特性。它由六个阳爻组成，象征着天数的六位，其中一位是阳爻，其他五位都是阴爻。

乾卦的卦辞是：“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。其中，元表示开始，意味着万事万物的起始；亨表示通，意味着道路畅通无阻；利表示有利于他人，关乎万事如意的信念；贞表示正，表示万事以天意为基准。

乾卦所蕴含的核心哲学是：阳刚之力量，观察世界需要力图建立自己的权威，同时要积极進取，具有強烈的進取心和竞争力。

乾卦的运筹和实践需要：

1. 初入职场：力图建立自己的权威，积极表现自己，得到领导的认可。

2. 事业发展：继续努力，抓住机遇，拓展事业领域。

3. 婚恋观：以 marriagewaiting 为止，寻找合适的伴侣。

4. 家庭生活：以家庭为主，关心家人，享受家庭的温馨。

5. 事业规划：积极拓展事业领域，不断挑战自我，实现自我价值。

6. 社交活动：积极参与社交，建立良好的人际关系。

7. 健康养生：保持良好的生活习惯，注重饮食和锻炼。

总结起来，乾卦要求人们积极进取，努力奋斗，以实现自己的目标和理想。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，象征刚健强劲的特性。它由两个阳爻（即表示数字1的爻）夹一个阴爻（即表示数字0的爻）组成，具有强烈的刚健强劲之力。乾卦的哲学意义是强调刚健强劲的特性，以及顺应天道的运行。

在卜筮中，乾卦预示着吉祥如意，事业顺利，权力威严，刚健强劲。然而，也需要注意防止过于刚硬，导致矛盾激化。在决策时，应该充分考虑各种因素，坚持正道，确保事业和生活的和谐发展。

乾卦的哲学内涵是刚健强劲，力量无穷，具有领导力和强大的力量。在决策中，要充分考虑各种因素，确保各种力量相互协调，促进事业和生活的和谐发展。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由两个卦象组合而成，外卦为坤，内卦为坎。坤为地，代表顺顺利利，而坎为水，代表变动，说明在这个过程中，将会有人文因素影响事物的发展。

根据《象辞》，地水师卦象征着天时地利，含括了雨泽滋润，及川流江海等情景。根据《断易天机》，地水师卦的寓言是：天时地利，相得相助，初得小利，后得大利。

根据运势学说，地水师卦具有顺利的形势，但需要注意避免初入险境，初始不宜轻动，初期不宜轻信别人的话，不宜 investing heavily or making important decisions. according to the traditional的解释, it's also said that the best course of action is to wait for the right moment, and that the best results come from waiting for the right moment.


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是周易中的一卦，由两个卦象组合而成，其中上卦是坤卦，下卦是坎卦。上卦坤卦代表地，下卦坎卦代表水，象征着地上的万物养育着水中生物，因此名为“地水师卦”。

在卜问中， this卦象代表了一定的吉祥，象征着地面上的生物都能得到滋养，同时也预示着可能会有困难，需要更加努力。


In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，卦象由六个阳爻（即阳性的木）排列而成，它们从下至上排列，看起来像是天上的云彩，因此得名“乾”。乾卦象征着刚强、坚毅和独立，以及与天地之间的联系。

乾卦的卦辞是：“乾为天，为父，为君，亲阳人。亲，先祖，家远的代际，宜先祖；国，先王，宜治国；夫，配偶，宜婚配；父，尊亲，宜尊父；君，朋友，宜友道。”卦辞描述了乾卦的内涵，它既象征着天，也象征着人。作为天，它与“先祖”和“先王”有关，表示宇宙的起点和万物的基础。作为人，它与“父”和“君”有关，表示人类的社会性和文化传承。

在《易经》中，乾卦是“八卦”之一，它既象征着阳，也象征着“刚”和“强”。乾卦的卦象既象征着“天”，也象征着“人”，它是“八卦”中最为重要的卦之一，表示宇宙的运行和万物的生成。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦，它的卦象是由两个阳爻夹一个阴爻构成，象征着天，也象征着刚强和坚毅。乾卦的卦辞是“元，亨，利，贞”，意味着“ beginning, success, profit, integrity”。它表示万事万物的本源和原理，是成功的基础。乾卦 also means "the heaven", and it is a symbol of the universe, with six lines of yin and one line of yang. It represents the power and strength of the world, and is a symbol of integrity.


In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦（师的卦象为乾为天，地为地）主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦是由天（乾）和地（坤）组成的卦象，乾为天，坤为地。师卦是《易经》中第八卦，属于坤卦（ earth）系列卦象，坤卦下卦为坤，上卦为乾，乾为天，坤为地。

师卦象征着师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦最佳状态为师的（师卦上卦为乾，下卦为坤），乾为天，坤为地，师卦象征师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

在占卦中，师卦卦象为吉凶相生，以阳刚为上，以顺为正，symbolism of the earth and sky，象征天地。师卦：师 师 师。三个师字，下坤上乾。

师卦卦象分析：师卦是乾卦下坤卦上，乾为天，坤为地，象征天和地。师卦象征师兵，主动向被动，主动而被动，地震师，吉。

师卦卦象占卜：占师卦，得师卦，吉。

师卦卦象应用：师卦象征师兵，主动向被动，主动而被动，象征天和地。适用于天和地的事物，占师卦 Creature and earth。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [21]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》中的第八卦，属于雷（卦头）坎（卦体）离（卦象）两种卦象的组合。讼卦象征诉讼、争议、争端等诉讼方面的现象。卦形为雷上坎下，雷象征刚毅，坎象征险，离象征烈。卦象下卦为坎，上卦为离，险险相间，刚毅相结合。

讼卦的卦辞：“讼，吉凶交战，得利藏损。”卦象下坎上离，坎险离光，光争下，险林光，刚毅争利，光损藏吉。

蔽卦（卦名：蔽）是《易经》中的第六卦，属于离（卦头）坤（卦体）震（卦象）两种卦象的组合。蔽卦象征屏蔽、遮蔽，阻止邪恶势力，弘扬正道。

蔽卦的卦辞：“蔽，利ulo，吉凶凶险皆屏蔽。”卦象下离上坤，离火遮蔽，坤地顺从，刚毅相合，和顺利害，吉祥安全。

 src="http://www.qk6.com/article/html/277776.html" target="_blank">http://www.qk6.com/article/html/277776.html


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
